You can test mistral 7B with this notebook on kaggle.

In [1]:
!pip install -qqq bitsandbytes
!pip install -qqq torch
!pip install  -qqq -U git+https://github.com/huggingface/transformers
!pip install -qqq -U git+https://github.com/huggingface/peft
!pip install -qqq -U git+https://github.com/huggingface/accelerate
!pip install -qqq datasets
!pip install -qqq loralib
!pip install -qqq einops
!pip install -qqq datasets
!pip install -qqq sentence-transformers

In [2]:
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset

In [3]:
# Load the model with bits and bytes config
# model = "GeneZC/MiniChat-3B"
model = "mistralai/Mistral-7B-Instruct-v0.1"
# model = "openai-community/gpt2-xl"

MODEL_NAME = model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    # device_map="cpu",
    trust_remote_code=True,
    quantization_config=bnb_config
)

# PEFT wrapper the model for training / fine-tuning
model = prepare_model_for_kbit_training(model)

# Define tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [4]:
# Load sentence transformer model
from sentence_transformers import SentenceTransformer
filter_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# Get vector from an input text by Sentence Transformer
def get_vector(text):
    text_vector = filter_model.encode(text)
    return torch.tensor(text_vector)

In [6]:
# Vectorization of "movie recommendation"
valid_query = "movie recommendation"
valid_vector = get_vector(valid_query)
valid_vector.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([384])

In [7]:
# Generate response based on a suitable prompt
def bot_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)

    # Generate answer
    output_sequences = model.generate(
        **inputs,
        max_length=1000,
        num_return_sequences=1,
        temperature=0.1,  # Adjust for creativity
        top_p=0.1,  # Nucleus sampling
        top_k=50,  # Top-k sampling
        no_repeat_ngram_size=2  # Prevent repeating n-grams
    )
    
    # Process the output to remove the input question from the response if it gets included
    answer = tokenizer.decode(output_sequences[0], skip_special_tokens=True).replace(prompt, "").strip()
    return answer

In [8]:
intentions = [
    "Can you recommend to me a movie similar to Harry Potter?",
    "What's the weather like today?",
    "Book a table for two at a Mexican restaurant tonight.",
    "Tell me a joke.",
    "How can I reset my password?",
    "Play some music from the 90s.",
    "What was the highest-grossing film in 2020?",
    "Find me a flight to New York next weekend.",
    "I need a recipe for a chocolate cake.",
    "What's the latest news on technology?",
    "Show me my photos from Paris.",
    "How do I make a Manhattan cocktail?",
    "Suggest a good workout for abs.",
    "What are some vegan restaurants nearby?",
    "Is there a good comedy movie to watch?",
    "I want to learn Python programming.",
    "Where can I watch The Office?",
    "Give me directions to the nearest gas station.",
    "What are the symptoms of COVID-19?",
    "Tell me about the life of Albert Einstein.",
    "Show me the latest horror films.",
    "What animated movies are recommended for kids?",
    "List all movies starring Tom Hanks.",
    "Who directed Pulp Fiction?",
    "Find movies similar to La La Land.",
    "What's the plot of The Matrix?",
    "Give me the soundtrack list from Guardians of the Galaxy.",
    "How many Oscars did Forrest Gump win?",
    "Is 'Joker' suitable for teenagers?",
    "What are some must-watch classic films?",
    "Play the trailer for 'Avengers: Infinity War'.",
    "I'm in the mood for a comedy. Any suggestions?",
    "What films are based on Stephen King's books?",
    "When was Schindler's List released?",
    "Find me a movie that will make me cry.",
    "What are the best sci-fi movies of the last decade?",
    "Which movies are in the Marvel Cinematic Universe?",
    "Tell me about the directorial style of Wes Anderson.",
    "How do I access movies on Netflix?",
    "Rate 'Mad Max: Fury Road' out of 10."
]

In [9]:
from torch.nn.functional import cosine_similarity

for intention in intentions:
    intention_vector = get_vector(intention)
    similarity = cosine_similarity(intention_vector.unsqueeze(0), valid_vector.unsqueeze(0))
    print(f"Question: {intention}")
    if similarity < 0.5:
        prompt = f"Answer the following question:\n{intention}"
        print(bot_response(prompt))
    if similarity >= 0.5:
        prompt = f"For the information mentioned, describe its genres and any notable themes:\n{intention}"
        bot_answer = bot_response(prompt)
        print(f"Answer:\n{bot_answer}")
    print("============================================================")
        

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: Can you recommend to me a movie similar to Harry Potter?


2024-03-16 00:23:07.295171: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 00:23:07.295275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 00:23:07.442624: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Answer:
Genres: Fantasy, Adventure, Drama
Notable themes include magic, friendship, loyalty, love, and good vs evil.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: How do I access movies on Netflix?
To access Netflix movies, you need to have a Netflix account. You can create an account on the Netflix website or through the app on your smart TV, smartphone, or tablet. Once you have an active account, log in and browse the vast collection of movies available on Netlfix.
You can search for specific movies or browe through different categories and genres. If you're not sure what to watch, Netflix also offers personalized recommendations based on movies you’ve watched and liked in the past. 
Once you find a movie you want to see, click on its title or image to open its page, where you can read more about it, watch a trailer, and see what other users have to say about the movie. To start watching the move, simply click the "Play" button.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: Rate 'Mad Max: Fury Road' out of 10.
Answer: 9.2
